<a href="https://colab.research.google.com/github/majdelhaj/ASK-FM-SCRAPER/blob/master/ask_fm_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

usernames = [
  'noorashkar',
  'esraarajab1',
  'yousefqattri',
  'AmeerAboArar',
  'ShorooqMaswadeh',
  'rafatamous',
  'bayanshaqour',
  'khaleelhoweidi',

  # 'ayah_herzallah',
  # 'wwwwwwww7',
  # 'mohammadghannam2001',
  # 'ashrfcov19',

];







def format_questions(questions):
    allQA = "";

    for q in questions:
        q_obj = BeautifulSoup(q.prettify(), 'html.parser')
        
        qaLine = "";
        qaLine += str(q_obj.find("h2").text).replace("\n", " ").strip() + "\t";
        try:
            qaLine += str(q_obj.find("div", {"class":"streamItem_content"}).text).replace("\n", " ").replace("View more", "").strip() + "\n";
        except Exception as e :
          try:
            qaLine += str(q_obj.find("img").text).replace("\n", " ").replace("View more", "").strip() + "\n";
          except Exception as e :
            continue;
        
        allQA += qaLine;
    return allQA;

def get_questions(user_name,top_answers=None):
    next_page = ""
    questions = ""
    total_answers_num = ""
    counter = 0
    base_link = "https://ask.fm"

    while True:
        total_link = ""
        if next_page:
            total_link = "{}/{}&no_prev_link=true".format(base_link,next_page)
        else:
            if top_answers : 
                total_link = "{}/{}/best".format(base_link,user_name)
            else:
                total_link = "{}/{}".format(base_link,user_name)
        print("Connecting to {}".format(total_link))
        r = requests.get(total_link)
        print(r.status_code)
        if r.status_code != 200:
            break
        else:
            print("Done with code {}".format(r.status_code))
            soup = BeautifulSoup(r.text, 'html.parser')
            if total_answers_num == "":
                total_answers_num = soup.find("div", {"class":"profileTabAnswerCount"}).text
            
            q = soup.find_all("article",{"class":"streamItem-answer"})
            counter += len(q)
            print("Done parsing {} of {} Answers".format(counter, total_answers_num))
            q = format_questions(q)
            questions += q;
            next_page = soup.find_all("a", {"class":"item-page-next"})

            if len(next_page) >0:
                next_page = next_page[0]["href"]
            else:
                break
    with open(user_name+'.txt', 'a') as f:
      f.write(questions)



for user_name in usernames:
  get_questions(user_name)